Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Ticker symbol': 'Symbol',
                      'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,Materials,Aluminum,"Pittsburgh, Pennsylvania"
ACHC,Acadia Healthcare,Health Care,Health Care Facilities,"Franklin, Tennessee"
ACM,AECOM,Industrials,Construction & Engineering,"Dallas, Texas"
ADC,Agree Realty,Real Estate,Retail REITs,"Bloomfield Hills, Michigan"
ADNT,Adient,Consumer Discretionary,Automotive Parts & Equipment,"Plymouth, Michigan"
...,...,...,...,...
X,U.S. Steel,Materials,Steel,"Pittsburgh, Pennsylvania"
XPO,"XPO, Inc.",Industrials,Cargo Ground Transportation,"Greenwich, Connecticut"
YETI,Yeti,Consumer Discretionary,Leisure Products,"Austin, Texas"


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp400 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp400.index = sp400.index.str.replace('.', '-', regex=False)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,US Stocks:Materials,Aluminum,"Pittsburgh, Pennsylvania"
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities,"Franklin, Tennessee"
ACM,AECOM,US Stocks:Industrials,Construction & Engineering,"Dallas, Texas"
ADC,Agree Realty,US Stocks:Real Estate,Retail REITs,"Bloomfield Hills, Michigan"
ADNT,Adient,US Stocks:Consumer Discretionary,Automotive Parts & Equipment,"Plymouth, Michigan"
...,...,...,...,...
X,U.S. Steel,US Stocks:Materials,Steel,"Pittsburgh, Pennsylvania"
XPO,"XPO, Inc.",US Stocks:Industrials,Cargo Ground Transportation,"Greenwich, Connecticut"
YETI,Yeti,US Stocks:Consumer Discretionary,Leisure Products,"Austin, Texas"


In [8]:
# Drop invalid symbols.
#sp400.drop(['AAXN'], inplace=True)

In [9]:
# Make symbols list.
symbols = list(sp400.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

AA ACHC ACM ADC ADNT AFG AGCO AIRC ALE ALGM ALLY ALV AM AMED AMG AMKR AN AR ARMK ARW ARWR ASB ASGN ASH ATR AVNT AVT AXTA AYI AZPN AZTA BC BCO BDC BERY BHF BJ BKH BLD BLKB BRBR BRKR BRX BURL BWXT BYD CABO CACI CADE CALX CAR CASY CBSH CBT CC CCK CDP CELH CFR CG CGNX CHDN CHE CHH CHK CHRD CHX CIEN CIVI CLF CLH CMC CNM CNO CNX CNXC COHR COKE COLB COLM COTY CPRI CR CRI CROX CRUS CSL CUBE CUZ CVLT CW CXT DAR DBX DCI DECK DINO DKS DLB DOC DOCS DT DTM EEFT EGP EHC ELS EME ENOV ENS EPR EQH ERIE ESAB ESNT ETRN EVR EWBC EXEL EXLS EXP EXPO FAF FBIN FCFS FCN FFIN FHI FHN FIVE FIX FLO FLR FLS FNB FND FNF FOXF FR FYBR G GATX GBCI GDDY GEF GGG GHC GLPI GME GMED GNTX GO GPK GPS GT GTLS GXO H HAE HALO HELE HGV HLI HOG HOMB HQY HR HRB HTZ HWC HXL IART IBKR IBOC IDA INGR IPGP IRDM IRT ITT JAZZ JEF JHG JLL JWN KBH KBR KD KEX KMPR KNF KNSL KNX KRC KRG LAD LAMR LANC LEA LECO LEG LFUS LII LITE LIVN LNTH LNW LOPE LPX LSCC LSTR M MAN MANH MASI MAT MDU MEDP MIDD MKSI MMS MORN MP MPW MSA MSM MTDR MTG MTN MTSI MTZ

In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AA,ACHC,ACM,ADC,ADNT,AFG,AGCO,AIRC,ALE,ALGM,...,WTFC,WTRG,WTS,WU,WWD,X,XPO,YETI,ZD,ZI
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,36.45,59.98,29.84,21.52,NaN,32.26,37.45,NaN,39.74,NaN,...,40.99,21.60,57.61,12.01,45.62,24.63,14.05,NaN,48.78,NaN
2015-01-05,34.34,59.12,28.53,21.76,NaN,31.84,35.73,NaN,39.11,NaN,...,40.18,21.18,56.14,11.90,44.63,23.48,13.62,NaN,47.99,NaN
2015-01-06,34.59,58.19,28.25,21.90,NaN,31.66,35.57,NaN,39.08,NaN,...,39.19,21.20,55.08,11.88,44.31,22.77,13.14,NaN,47.59,NaN
2015-01-07,35.49,60.63,28.83,22.15,NaN,31.87,35.54,NaN,39.81,NaN,...,39.53,21.39,54.67,11.76,44.73,22.82,13.19,NaN,47.78,NaN
2015-01-08,36.50,61.76,29.72,22.23,NaN,32.44,36.30,NaN,40.60,NaN,...,39.85,21.53,55.24,12.00,44.50,23.32,13.26,NaN,48.72,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,33.77,77.86,92.09,62.27,36.74,118.67,122.28,33.86,61.63,30.90,...,92.88,37.14,209.99,11.84,135.11,47.97,89.51,51.86,67.98,18.68
2023-12-26,33.87,78.79,92.24,62.79,36.90,118.82,122.58,34.33,61.69,31.20,...,94.31,37.42,212.01,11.75,135.40,48.35,90.12,52.82,68.27,18.93
2023-12-27,34.81,78.68,92.55,62.74,36.53,119.01,122.31,34.68,61.34,31.09,...,94.09,37.40,211.31,11.75,136.03,48.61,89.92,53.60,68.25,18.81


In [12]:
# Sample symbol.
symbol = 'ACHC'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

-4.670831890214533

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.28704638956469347

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.17122946810870746

In [17]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [18]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.09
2015-03-31    0.13
2015-04-30   -0.04
2015-05-31    0.08
              ... 
2023-08-31   -0.02
2023-09-30   -0.09
2023-10-31    0.05
2023-11-30   -0.01
2023-12-31    0.07
Freq: M, Name: ACHC, Length: 108, dtype: float64

In [19]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.26237493809303813

In [20]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [21]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [22]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')